In [ ]:
import os
import time
import json
from copy import copy

import pandas as pd
import gokart
import luigi
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from mahjong.shanten import Shanten
from mahjong.hand_calculating.hand import HandCalculator
from mahjong.tile import TilesConverter
from mahjong.hand_calculating.hand_config import HandConfig
from mahjong.meld import Meld

from mleagueloader import MLEAGUELoader, format_raw_data, tehai_shanten, TehaiGenerator

In [ ]:
DEBUG = True

In [ ]:
webdriver.__version__

In [ ]:
pd.options.display.max_rows = 200
pd.options.display.max_colwidth=80

In [ ]:
load_dotenv()

MLEAGUE_EMAIL = os.environ['MLEAGUE_EMAIL']
MLEAGUE_PASS=os.environ['MLEAGUE_PASS']

In [ ]:
class LoaderTask(gokart.TaskOnKart):
    season = luigi.IntParameter(default=2022)
    debug = luigi.BoolParameter(default=True)

    def run(self):
        options = Options()

        # Headless
        options.headless = False

        # ブラウザ起動
        driver = webdriver.Chrome(executable_path="chromedriver.exe", options=options)
        loader = MLEAGUELoader(driver=driver)

        loader.login_fun(MLEAGUE_EMAIL, MLEAGUE_PASS)

        # 2022シーズンの終了した試合を取得
        schedule = loader.get_match_schedule(self.season)
        schedule.sort(reverse=False)

        if self.debug:
            schedule = schedule[:2]

        df = loader.load(schedule)
        self.dump(df)

In [ ]:
df = gokart.build(LoaderTask())

In [ ]:
game_df, user_df = format_raw_data(df)

In [ ]:
game_df.head(200)

In [ ]:
game_df.dtypes

In [ ]:
user_df.head(100)

In [ ]:
tehaigen = TehaiGenerator()
game_df = tehaigen.generate(game_df)
game_df.head(300)[['player', 'cmd', 'hai', 'tehai']]

In [ ]:
game_df['tehai_shanten'] = game_df['tehai'].apply(lambda x: tehai_shanten(x))

game_df.head(300)